# MapReduce 1/2

Dans cette séance vous allez utiliser les fonctions `map()` et `functools.reduce()` de Python pour illustrer les concepts du modèle de programmation MapReduce.

## Exercice #1 : similarité cosinus avec les fonctions `map` et `functools.reduce`

Le but de cet exercice est de comprendre le fonctionnement des fonctions `map` et `functools.reduce` de Python et de les appliquer au calcul de la [similarité cosinus](https://fr.wikipedia.org/wiki/Similarit%C3%A9_cosinus) entre deux vecteurs.

### Processus Map

**Question 1.1**

Ecrire et tester une fonction `carre_liste()` qui prend une liste de nombres en entrée et renvoie la liste des carrés de ces nombres en utilisant la fonction `carre()` fournie et une liste en compréhension. Afficher le résultat obtenu.

In [32]:
def carre(x):
    return x**2

def carre_liste(la_liste):
    return [carre(x) for x in la_liste]

ma_liste1 = list(range(1,10))
resultat_1_1 = carre_liste(ma_liste1)
print(resultat_1_1)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


**Question 1.2**

Ecrire une instruction qui fait la même chose en utilisant la fonction `map()` de python. Afficher le résultat obtenu.

*Indication : la fonction `map()` renvoie un itérateur et non une liste. On peut cependant obtenir la liste correspondante en applicant un "cast" à cet itérateur à l'aide de la fonction `list()`*

In [33]:
resultat_1_2 = map(carre, ma_liste1)   # itérateur

liste_carres = list(resultat_1_2)    # liste correspondante
print(liste_carres)

[1, 4, 9, 16, 25, 36, 49, 64, 81]


**Question 1.3**

Ecrire une fonction `prod_listes()` qui prend en entrée 2 listes de nombres de même taille et qui renvoie une liste des produits 2 à 2 de ces nombres en utilisant la fonction `produit()` fournie et une liste en compréhension. Afficher le résultat obtenu.

*Indication : pour une meilleure implémentation, penser à utiliser la fonction [`zip()`](https://docs.python.org/3/library/functions.html#zip) de Python*

In [34]:
def produit(x,y):
    return x*y

def prod_listes(liste1, liste2):
    return [produit(x,y) for x,y in zip(liste1, liste2)]

ma_liste2 = list(range(9,0,-1))

resultat_1_3 = prod_listes(ma_liste1, ma_liste2)
print(resultat_1_3)

[9, 16, 21, 24, 25, 24, 21, 16, 9]


**Question 1.4**

Ecrire une instruction qui fait la même chose en utilisant la fonction `map()` de python. Afficher le résultat obtenu.

In [35]:
resultat_1_4 = map(produit, ma_liste1, ma_liste2)
print(list(resultat_1_4))

[9, 16, 21, 24, 25, 24, 21, 16, 9]


### Processus Map+Reduce

**Question 1.5**

Ecrire une fonction **recursive** `reduce_liste()` qui prend en premier paramètre le nom d'une fonction d'opération (telle que `addition` fournies ci-dessous, ou `produit` fournies ci-dessus) et en deuxième paramètre une liste de nombres, et qui permet d'appliquer l'opération de façon récursive à l'ensemble des nombres de la liste. 

Exemple : si `f()` est la fonction d'opération et `x1,x2,x3,x4` les éléments de la liste, l'appel de la fonction récursive sur cette liste doit renvoyer le résultat final suivant :

`f(f(f(x1, x2), x3), x4)`

On prendra soin de bien identifier le dernier calcul de la récursion (c.a.d celui qui arrête les appels récursifs).

Tester cette fonction sur la liste de carrés obtenue à la question 1.2 et afficher le résultat.

In [36]:
from functools import reduce

def addition(x,y):
    return x+y


def reduce_liste(operation, la_liste):
    return operation(reduce_liste(operation, la_liste[:-1]), la_liste[-1]) if len(la_liste)> 2 else operation(la_liste[0], la_liste[1])

resultat_1_5 = reduce_liste(addition, liste_carres)
print(resultat_1_5)

285


**Question 1.6**

Ecrire une instruction unique imbriquant les fonctions `map()`, `reduce()` (du module `functools`), `carre()` et `addition()` (fournies ci-dessus) permettant de calculer la somme des carrés d'une liste. Afficher le résultat obtenu.

In [37]:
from functools import reduce

resultat = reduce(addition, map(carre, ma_liste1))
print(resultat)

285


*Remarque : au lieu de définir nous même la fonction `addition()` on peut passer en premier paramètre à `reduce()` la fonction `add` du module `operator` de Python.*

**Question 1.6**

Ecrire une instruction unique imbriquant les fonctions `map()`, `reduce()`, `produit()` (fournie ci-dessus) et la fonction `add()` du module `operator` permettant de calculer la somme des produits 2 à 2 des éléments de deux listes de même taille. Afficher le résultat obtenu.

In [38]:
from operator import add

resultat = reduce(add, map(produit, ma_liste1, ma_liste2))
print(resultat)

165


*Remarque : comme pour l'addition, au lieu de définir la fonction `produit()` on peut utiliser la fonction `mul()` du module `operator`.*

**Question 1.7**

Ecrire une fonction `sim_cos()` prenant en entrée deux listes de nombres de même taille représentant deux vecteurs, et qui calcule la similarité cosinus entre ces deux vecteurs.

In [39]:
from math import sqrt
from operator import mul

def sim_cos(v1, v2):
    norm1 = reduce(add, map(carre,v1))
    norm2 = reduce(add, map(carre,v2))
    prod = reduce(add, map(mul, v1, v2))

    return prod/sqrt(norm1*norm2)


resultat = sim_cos(ma_liste1, ma_liste2)
print(resultat)

0.5789473684210527


## Exercice #2 : wordcount avec les fonctions `map` et `functools.reduce`

Le but de l'exercice est d'écrire un programme de comptage d'occurences de mots dans un corpus de texte (réparti en plusieurs fichiers) en utilisant les fonctions `map` et `functools.reduce` de Python. Les fichiers de texte seront générés automatiquement grâce au module `lorem` (génération de faux textes en latin).

Le prétraitement des textes sera ici très basique : on passera le texte en minuscule et on retirera les points de ponctuattion.

In [40]:
import lorem

data_in = []

# création des fichiers texte et de la liste de ces fichiers
for i in range(3):
    with open(f"sample{i}.txt", "w", encoding='utf-8') as f:
        for j in range(2):
            f.write(lorem.text())
    data_in.append(f"sample{i}.txt")

print(data_in)

['sample0.txt', 'sample1.txt', 'sample2.txt']


**Question 2.1**

Ecrire une fonction `tokenise()` qui prend en entrée un texte, qui pré-traite ce texte et renvoie une liste des mots du texte. Tester sur un des fichiers texte créés.

In [41]:
def tokenise(text):
    return text.lower().replace(".", "").split()

with open("sample0.txt", 'r', encoding='utf-8') as i_file:
    content = i_file.read()

res_tok = tokenise(content)
print(res_tok)

['quaerat', 'est', 'ipsum', 'non', 'tempora', 'numquam', 'quaerat', 'ipsum', 'dolor', 'adipisci', 'labore', 'numquam', 'dolor', 'dolor', 'sed', 'consectetur', 'aliquam', 'tempora', 'quisquam', 'porro', 'labore', 'labore', 'quisquam', 'quiquia', 'ipsum', 'voluptatem', 'modi', 'etincidunt', 'est', 'quiquia', 'tempora', 'neque', 'consectetur', 'quaerat', 'eius', 'labore', 'adipisci', 'quisquam', 'porro', 'velit', 'labore', 'ut', 'dolor', 'labore', 'sit', 'adipisci', 'amet', 'sed', 'aliquam', 'ut', 'neque', 'amet', 'modi', 'sed', 'quisquam', 'sed', 'est', 'eius', 'numquam', 'est', 'aliquam', 'amet', 'velit', 'quiquia', 'neque', 'dolore', 'adipisci', 'adipisci', 'labore', 'amet', 'etincidunt', 'modi', 'porro', 'numquam', 'tempora', 'dolorem', 'neque', 'non', 'neque', 'adipisci', 'modi', 'ut', 'quaerat', 'labore', 'est', 'quaerat', 'dolorem', 'aliquam', 'dolore', 'numquam', 'dolorem', 'aliquam', 'eius', 'quaerat', 'non', 'ipsum', 'amet', 'quaerat', 'tempora', 'eius', 'non', 'sit', 'ipsum', '

**Question 2.2**

Ecrire une fonction `combine_words()` qui prend en entrée une liste de mot et renvoie le nombre d'occurences de chaque mot unique rencontré sous forme d'une liste de tuples `(word, nb_word)`. Tester sur le résultat précédent.

In [48]:
def combine_words(word_list):
    counts = {}
    for w in word_list :
        counts[w] = 1 if w not in counts else counts[w] + 1
        
    return list(counts.items())

print(combine_words(res_tok))

[('quaerat', 24), ('est', 15), ('ipsum', 18), ('non', 12), ('tempora', 19), ('numquam', 20), ('dolor', 16), ('adipisci', 22), ('labore', 30), ('sed', 14), ('consectetur', 13), ('aliquam', 20), ('quisquam', 13), ('porro', 14), ('quiquia', 11), ('voluptatem', 23), ('modi', 21), ('etincidunt', 15), ('neque', 20), ('eius', 15), ('velit', 12), ('ut', 13), ('sit', 18), ('amet', 25), ('dolore', 19), ('dolorem', 13), ('magnam', 11), ('temporanumquam', 1)]


**Question 2.3**

Ecrire une fonction `tokenise_and_combine_words()` qui prend en entrée un nom de fichier texte et qui utilise les deux fonctions précédentes pour fournir en sortie une liste des tuples `(word, nb_word)` indiquant le nombre d'occurence de chaque mot rencontré dans le fichier. Tester sur une des fichiers texte.

In [51]:
def tokenise_and_combine_words(filename):
    with open(filename, 'rt', encoding='utf-8') as ifile:
        content = ifile.read()
    
    return combine_words(tokenise(content))

tokenise_and_combine_words('sample0.txt')[:5]

[('quaerat', 24), ('est', 15), ('ipsum', 18), ('non', 12), ('tempora', 19)]

**Question 2.4**

Ecrire une instruction utilisant la fonction `map` de python permettant d'obtenir une listes des liste de tuples `(word, nb_word)` obtenues sur chaque fichier texte du corpus (ou plus précisément un itérateur sur cette liste). Afficher le résultat.

In [55]:
map_res = map(tokenise_and_combine_words, data_in)
# print(list(map_res))           # Attention : ligne à commenter si on veut pouvoir utiliser l'itérateur map par la suite
                                 # (le fait de créer une liste avec l'itérateur épuise cet itérateur)

**Question 2.5**

Ecrire une fonction `group_counts()` qui prendra en entrée une liste des listes de tuples `(word, nb_word)` issues des différents processus Map (ou des itérateurs sur ces listes) et qui fournira en sortie une liste de tuples `(word, list_nb_word)` où `list_nb_word` est une liste des occurences du mot `word` dans les différents fichiers textes.
*Exemple de sortie de la fonction : `[('numquam', [26, 17, 16]), ('quiquia', [21, 13, 26]), ('ipsum', [19, 14, 19]),... ]`*

In [56]:
def group_counts(list_of_lists):
    counts = {}
    for l in list_of_lists:
        for w,n in l : 
            counts[w] = [n] if w not in counts else counts[w]+[n]
    return list(counts.items())

group_res =  group_counts(map_res)
print(group_res)

[('quaerat', [24, 9, 14]), ('est', [15, 13, 16]), ('ipsum', [18, 16, 16]), ('non', [12, 12, 13]), ('tempora', [19, 16, 13]), ('numquam', [20, 15, 17]), ('dolor', [16, 10, 21]), ('adipisci', [22, 10, 16]), ('labore', [30, 18, 20]), ('sed', [14, 12, 13]), ('consectetur', [13, 15, 12]), ('aliquam', [20, 12, 14]), ('quisquam', [13, 12, 10]), ('porro', [14, 27, 13]), ('quiquia', [11, 21, 15]), ('voluptatem', [23, 12, 26]), ('modi', [21, 18, 19]), ('etincidunt', [15, 18, 12]), ('neque', [20, 16, 17]), ('eius', [15, 14, 23]), ('velit', [12, 14, 15]), ('ut', [13, 13, 14]), ('sit', [18, 3, 5]), ('amet', [25, 11, 19]), ('dolore', [19, 13, 12]), ('dolorem', [13, 10, 16]), ('magnam', [11, 16, 1]), ('temporanumquam', [1]), ('consecteturdolorem', [1]), ('nequeconsectetur', [1])]


**Question 2.6**

Ecrire une fonction `cumulate_counts()` qui prendra en entrée un tuple `(word, list_nb_word)` et qui utilisera la fonction `functools.reduce()` pour renvoyer un tuple `(word, tot_nb_word)` où `tot_nb_word` est le nombre d'occurences du mot `word` cumulées sur l'ensemble du corpus. Tester sur le premier élément de la liste obtenue à la question précédente.

In [57]:
def cumulate_counts(word_listnbword):
    word, list_nb_word = word_listnbword
    return (word, reduce(add, list_nb_word))

print(cumulate_counts(group_res[0]))

('quaerat', 47)


**Question 2.7**

Ecrire une instruction utilisant la fonction `map()` qui permet d'obtenir une liste de tuples `(word, tot_nb_word)` sur l'ensemble du corpus. Afficher le résultat obtenu par ordre décroissant du nombre d'occurences de mot.

In [58]:
res_reduce = map(cumulate_counts, group_res)
print(sorted(res_reduce, key=lambda x : x[1], reverse=True))

[('labore', 68), ('voluptatem', 61), ('modi', 58), ('amet', 55), ('porro', 54), ('neque', 53), ('numquam', 52), ('eius', 52), ('ipsum', 50), ('tempora', 48), ('adipisci', 48), ('quaerat', 47), ('dolor', 47), ('quiquia', 47), ('aliquam', 46), ('etincidunt', 45), ('est', 44), ('dolore', 44), ('velit', 41), ('consectetur', 40), ('ut', 40), ('sed', 39), ('dolorem', 39), ('non', 37), ('quisquam', 35), ('magnam', 28), ('sit', 26), ('temporanumquam', 1), ('consecteturdolorem', 1), ('nequeconsectetur', 1)]
